# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [2]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
import data_generator as dg
import simple_mdp as sm

In [3]:
n_concepts = 5
concept_tree = sm.create_custom_dependency()
n_students = 100000
seqlen = 10
policy = 'random'
filename = "test.pickle".format(n_students, seqlen, policy)

In [75]:
print ("Initializing synthetic data sets...")
dg.generate_data(concept_tree, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 10.
Data generation completed. 


In [11]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [5]:
# test_model hidden=16
# test_model_small hidden=5
dmodel = dmc.DynamicsModel(model_id="test_model_small", timesteps=seqlen, load_checkpoint=True)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model_small/
Checkpoint directory path: ../checkpoints/test_model_small/
Checkpoint filename: /usr0/home/zguo/Documents/smart-tutor/checkpoints/test_model_small/_-28140
INFO:tensorflow:Restoring parameters from /usr0/home/zguo/Documents/smart-tutor/checkpoints/test_model_small/_-28140
Checkpoint loaded.
Model loaded.


In [14]:
# load toy data
data = dataset_utils.load_data(filename="../synthetic_data/test.pickle")
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [15]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])

In [18]:
dmodel.train(train_data, n_epoch=5, load_checkpoint=True)

Training Step: 28139  | total loss: 0.01657 | time: 20.252s
| Adam | epoch: 020 | loss: 0.01657 -- iter: 89984/90000
Training Step: 28140  | total loss: 0.01662 | time: 21.461s
| Adam | epoch: 020 | loss: 0.01662 | val_loss: 0.01688 -- iter: 90000/90000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [6]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

NameError: name 'input_data_' is not defined

In [12]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [13]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [14]:
rnnsim.advance_simulator(act, 1)


['MODELS_DICT_PATH', 'N_CONCEPTS', 'RNN_DATA_DIR', 'SYN_DATA_DIR', 'StudentAction', '__builtins__', '__doc__', '__file__', '__name__', '__package__', 'absolute_import', 'convert_to_rnn_input', 'division', 'load_data', 'load_rnn_data', 'np', 'pickle', 'preprocess_data_for_rnn', 'print_function', 'save_rnn_data', 'tf']


In [15]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.7310570478439331, 0.5059529542922974, 0.49991777539253235, 0.018891867250204086, 0.033908743411302567]
1.78972838819
